Use this utlity to update the returns and std_dev fields within investment-options.csv

In [1]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import brownbear as bb

# Format price data.
pd.options.display.float_format = '{:0.2f}'.format

%matplotlib inline

In [3]:
# Set size of inline plots.
'''note: rcParams can't be in same cell as import matplotlib
   or %matplotlib inline
   
   %matplotlib notebook: will lead to interactive plots embedded within
   the notebook, you can zoom and resize the figure
   
   %matplotlib inline: only draw static images in the notebook
'''
plt.rcParams["figure.figsize"] = (10, 7)

Globals

In [4]:
# Set refresh_timeseries=True to download timeseries.  Otherwise /symbol-cache is used.
refresh_timeseries = True

In [5]:
# Read in investment options input csv.
etfs = pd.read_csv('investment-options-in.csv', skip_blank_lines=True, comment='#')
etfs = list(etfs['Investment Option'])

In [6]:
# Eliminate repeat symbols.
symbols = set(etfs)

In [7]:
# Get the timeseries for the symbols and compile into a single csv.
bb.fetch_timeseries(symbols, refresh=refresh_timeseries)
bb.compile_timeseries(symbols)

IWD VCIT XLI SLV XLC QQQ VTI BNDX UGA XLE IWF PALL XOP OIH XLRE EBND BWX DBA FXE EEM FXA DBB LQD XLV DBP XLU FXB XLF DIA SHY TLT JPST CANE BND SDY FXF FXC NOBL SPY XLP AGG IBND CPER PPLT VNQ VDE VPL FXY EFA VXUS JNK USO CORN DBE SLY XLY XLB IWM PFF BWZ XLK BIL SHV UNG CYB IYR RSP GDX PGX GLD IEUR UUP MDY VCSH 


In [8]:
# Read symbols timeseries into a dataframe.
df = pd.read_csv('symbols-timeseries.csv', skip_blank_lines=True, comment='#')
df.set_index("Date", inplace=True)

In [9]:
# Sample symbol.
symbol = 'SPY'

In [10]:
annual_returns = bb.annualized_returns(df, timeperiod='daily', years=1)
# Sample
annual_returns[symbol]

-6.492654731289694

In [11]:
# Calculate 1 month, 3 months, 1 year, 3 year, and 5 year annualized returns.
annual_returns_1mo = bb.annualized_returns(df, timeperiod='daily', years=1/12)
annual_returns_3mo = bb.annualized_returns(df, timeperiod='daily', years=3/12)
annual_returns_1yr = bb.annualized_returns(df, timeperiod='daily', years=1)
annual_returns_3yr = bb.annualized_returns(df, timeperiod='daily', years=3)
annual_returns_5yr = bb.annualized_returns(df, timeperiod='daily', years=5)

In [12]:
# Calculate 20 day annualized volatility.
daily_returns = df.pct_change()
years = bb.TRADING_DAYS_PER_MONTH / bb.TRADING_DAYS_PER_YEAR
vola = bb.annualized_standard_deviation(daily_returns, timeperiod='daily', years=years)
vola[symbol]

0.1651757004462286

In [13]:
# Calculate 20 day annualized downside volatility.
ds_vola = bb.annualized_standard_deviation(daily_returns, timeperiod='daily', years=years, downside=True)
ds_vola['SPY']

0.0767300205982728

In [14]:
# Resample df on a monthly basis.
df.index = pd.to_datetime(df.index)
monthly = df.resample('M').ffill()
bb.print_full(monthly[symbol])

Date
2015-01-31   172.30
2015-02-28   181.98
2015-03-31   179.12
2015-04-30   180.88
2015-05-31   183.21
2015-06-30   179.49
2015-07-31   183.54
2015-08-31   172.36
2015-09-30   167.96
2015-10-31   182.25
2015-11-30   182.91
2015-12-31   179.75
2016-01-31   170.80
2016-02-29   170.66
2016-03-31   182.14
2016-04-30   182.86
2016-05-31   185.97
2016-06-30   186.61
2016-07-31   193.42
2016-08-31   193.65
2016-09-30   193.66
2016-10-31   190.31
2016-11-30   197.32
2016-12-31   201.32
2017-01-31   204.92
2017-02-28   212.97
2017-03-31   213.24
2017-04-30   215.35
2017-05-31   218.39
2017-06-30   219.78
2017-07-31   224.30
2017-08-31   224.96
2017-09-30   229.49
2017-10-31   234.90
2017-11-30   242.08
2017-12-31   245.01
2018-01-31   258.82
2018-02-28   249.41
2018-03-31   242.57
2018-04-30   243.83
2018-05-31   249.76
2018-06-30   251.19
2018-07-31   260.50
2018-08-31   268.81
2018-09-30   270.41
2018-10-31   251.72
2018-11-30   256.39
2018-12-31   233.82
2019-01-31   252.54
2019-02-28   26

In [15]:
# Calculate monthly returns.
monthly_returns = monthly.pct_change()
monthly_returns[symbol]

Date
2015-01-31     nan
2015-02-28    0.06
2015-03-31   -0.02
2015-04-30    0.01
2015-05-31    0.01
              ... 
2022-10-31    0.08
2022-11-30    0.06
2022-12-31   -0.06
2023-01-31    0.06
2023-02-28    0.03
Freq: M, Name: SPY, Length: 98, dtype: float64

In [16]:
# Calculate 3 year annualized standard deviation.
std_dev = bb.annualized_standard_deviation(monthly_returns, timeperiod='monthly', years=3)
std_dev[symbol]

0.20419318295884786

In [17]:
# Read investment-options-in.csv
lines = []
with open('investment-options-in.csv', 'r') as f:
    lines = [line.strip() for line in f]
#lines

In [18]:
# For each etf, write out the 1 Yr, 3 Yr, 5 Yr, and std dev.
out = []
for line in lines:
    # Copy empty and comment lines.
    if not line or line.startswith('#'):
        out.append(line)
        continue
    # Split line by comma delimiter; strip the fields.
    fields = line.split(',')
    fields = [field.strip() for field in fields]
    symbol = fields[0].strip('\"')
    # Copy header.
    if symbol == 'Investment Option':
        out.append(line)
        continue
    ret_1mo = annual_returns_1mo[symbol]
    ret_3mo = annual_returns_3mo[symbol]
    ret_1yr = annual_returns_1yr[symbol]
    ret_3yr = annual_returns_3yr[symbol]
    ret_5yr = annual_returns_5yr[symbol]
    
    if np.isnan(ret_3yr): ret_3yr = ret_1yr
    if np.isnan(ret_5yr): ret_5yr = ret_3yr

    _vola = vola[symbol]*100
    _ds_vola = ds_vola[symbol]*100
    sd = std_dev[symbol]*100

    out.append('{},{},{},"{:0.2f}","{:0.2f}","{:0.2f}","{:0.2f}","{:0.2f}","{:0.2f}","{:0.2f}","{:0.2f}"'
               .format(fields[0], fields[1], fields[2],
                       ret_1mo, ret_3mo, ret_1yr, ret_3yr, ret_5yr, _vola, _ds_vola, sd)) 

In [19]:
# Write out asset-classes.csv
with open('investment-options.csv', 'w') as f:
    for line in out:
        f.write(line + '\n')